In [ ]:
# pip install pydub

In [8]:
import os
import random
import numpy as np
from pydub import AudioSegment
from pydub.generators import WhiteNoise

# PinkNoise và BrownNoise không hỗ trợ trong pydub mặc định, tạo giả lập bằng cách lọc white noise

def generate_pink_noise(duration_ms):
    white = WhiteNoise().to_audio_segment(duration=duration_ms)
    return white.low_pass_filter(2000)  # Giả lập pink noise bằng lọc tần số thấp

def generate_brown_noise(duration_ms):
    white = WhiteNoise().to_audio_segment(duration=duration_ms)
    return white.low_pass_filter(500)  # Giả lập brown noise bằng lọc tần số thấp mạnh hơn

def generate_random_noise(duration_ms, noise_type="white"):
    """
    Sinh noise ngẫu nhiên dựa trên loại noise và độ dài.
    
    :param duration_ms: Độ dài của noise cần tạo (ms).
    :param noise_type: Loại noise (white, pink, brown).
    :return: AudioSegment chứa noise.
    """
    if noise_type == "white":
        return WhiteNoise().to_audio_segment(duration=duration_ms)
    elif noise_type == "pink":
        return generate_pink_noise(duration_ms)
    elif noise_type == "brown":
        return generate_brown_noise(duration_ms)
    else:
        return WhiteNoise().to_audio_segment(duration=duration_ms)  # Mặc định white noise

def add_noise_to_audio(original_audio_path, noise_audio_path, output_path):
    """
    Thêm nhiễu vào file audio gốc, gồm noise ngẫu nhiên và noise từ file nhỏ.
    Lưu file kết quả vào folder mới, không ghi đè.

    :param original_audio_path: Đường dẫn đến file audio gốc (mp3).
    :param noise_audio_path: Đường dẫn đến file audio nhiễu (wav).
    :param output_path: Đường dẫn lưu file kết quả (mp3).
    """
    # Load audio gốc và noise từ file nhỏ
    original_audio = AudioSegment.from_mp3(original_audio_path)
    noise_file_audio = AudioSegment.from_wav(noise_audio_path)

    # Sinh noise ngẫu nhiên với độ dài khớp audio gốc
    noise_type = random.choice(["white", "pink", "brown"])
    random_noise = generate_random_noise(len(original_audio), noise_type=noise_type)

    # Ghép noise từ file nhỏ để đủ dài
    if len(noise_file_audio) < len(original_audio):
        noise_file_audio = noise_file_audio * (len(original_audio) // len(noise_file_audio) + 1)
    noise_file_audio = noise_file_audio[:len(original_audio)]

    # Điều chỉnh âm lượng nhiễu nhỏ hơn âm gốc 1/4
    original_rms = original_audio.rms

    for noise_audio in [random_noise, noise_file_audio]:
        noise_rms = noise_audio.rms
        if noise_rms > 0:
            gain_db = 20 * np.log10(original_rms / 4 / noise_rms)
            noise_audio = noise_audio.apply_gain(gain_db)
        # Trộn vào original
        original_audio = original_audio.overlay(noise_audio)

    # Lưu file kết quả vào output_path
    original_audio.export(output_path, format="mp3")

def process_audio_lists(original_audio_folder, noise_audio_folder, output_folder):
    """
    Thêm noise ngẫu nhiên và noise từ file nhỏ vào tất cả file mp3 gốc.
    Lưu kết quả vào folder mới, không ghi đè file gốc.

    :param original_audio_folder: Thư mục chứa audio gốc.
    :param noise_audio_folder: Thư mục chứa audio nhiễu nhỏ.
    :param output_folder: Thư mục để lưu file kết quả.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    original_files = [f for f in os.listdir(original_audio_folder) if f.endswith(".mp3")]
    noise_files = [f for f in os.listdir(noise_audio_folder) if f.endswith(".wav")]

    for original_file in original_files:
        original_path = os.path.join(original_audio_folder, original_file)
        noise_file = random.choice(noise_files)
        noise_path = os.path.join(noise_audio_folder, noise_file)

        output_path = os.path.join(output_folder, original_file)
        add_noise_to_audio(original_path, noise_path, output_path)
        print(f"Processed {original_file} with noise {noise_file} -> {output_path}")

# Ví dụ sử dụng:
# process_audio_lists("path_to_original_audio", "path_to_noise_audio", "path_to_output_folder")

In [9]:
process_audio_lists("./SampleAudioClean/data", "./SampleAudioClean/noise", "./add_noise_output")

Processed FPTOpenSpeechData_Set001_V0.1_000925.mp3 with noise 1-43382-A-1.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000925.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000151.mp3 with noise 1-63871-A-10.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000151.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000718.mp3 with noise 1-51436-A-17.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000718.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000394.mp3 with noise 5-252248-A-34.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000394.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000493.mp3 with noise 4-152995-A-24.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000493.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000237.mp3 with noise 1-17092-B-27.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000237.mp3
Processed FPTOpenSpeechData_Set001_V0.1_000859.mp3 with noise 1-39901-A-11.wav -> ./add_noise_output/FPTOpenSpeechData_Set001_V0.1_000859.mp3
Proce